In [3]:
#!pip install opendatasets
import opendatasets as od
import pandas
import os

In [2]:
od.download("https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia")

100%|██████████| 2.29G/2.29G [04:25<00:00, 9.27MB/s]


In [36]:
import stat

def onerror(func, path, exc_info):
    """
    Error handler for ``shutil.rmtree``.

    If the error is due to an access error (read only file)
    it attempts to add write permission and then retries.

    If the error is for another reason it re-raises the error.
    
    Usage : ``shutil.rmtree(path, onerror=onerror)``
    """
    # Is the error an access error?
    if not os.access(path, os.W_OK):
        os.chmod(path, stat.S_IWUSR)
        func(path)
    else:
        raise

In [41]:
import shutil

current_dir = os.getcwd()

dir_path1 = 'chest-xray-pneumonia/chest_xray/train'
dir_path2 = '/chest-xray-pneumonia/chest_xray/chest_xray'
absolute_path = os.path.join(current_dir, dir_path1)
absolute_path
#shutil.rmtree(absolute_path, onerror=onerror)


'c:\\Users\\clash\\Desktop\\4\\TFG\\Secure-FL-Flow\\envoy\\chest-xray-pneumonia/chest_xray/train'

In [49]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms

current_dir = os.getcwd()

tr_path = 'chest-xray-pneumonia/chest_xray/train'
val_path = 'chest-xray-pneumonia/chest_xray/val'
te_path = 'chest-xray-pneumonia/chest_xray/test'

train_path = os.path.join(current_dir, tr_path)
valid_path = os.path.join(current_dir, val_path)
test_path = os.path.join(current_dir, te_path)


train_transform = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.ColorJitter(brightness=0.2, contrast=0, saturation=0, hue=0),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.Resize((150,150)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))])

val_test_transform = transforms.Compose([
                  transforms.Resize((150,150)),
                  transforms.Grayscale(),
                  transforms.ToTensor(),
                  transforms.Normalize((0.5), (0.5))])

train_dataset = datasets.ImageFolder((train_path), transform=train_transform)
val_dataset = datasets.ImageFolder((valid_path), transform=val_test_transform)
test_dataset = datasets.ImageFolder((test_path), transform=val_test_transform)

# Create data loaders to load the data in batches
val_test_data = torch.utils.data.ConcatDataset([val_dataset, test_dataset])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None, shuffle=True)
val_test_loader = torch.utils.data.DataLoader(val_test_data, batch_size=None, shuffle=False)

# Extract the images and labels from the data loaders
x_train, y_train = zip(*[(x, y) for x, y in train_loader])
x_test, y_test = zip(*[(x, y) for x, y in val_test_loader])